In [ ]:
# train_lstm_plank.ipynb

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import os

# -----------------------------
# 1) Charger les séquences
# -----------------------------
data = np.load("plank_model/data/plank_sequences.npz")
X = data["X"]
y = data["y"]

print("X shape:", X.shape)
print("y shape:", y.shape)

# -----------------------------
# 2) Créer le DataLoader PyTorch
# -----------------------------
batch_size = 32
dataset = TensorDataset(torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long))
train_size = int(len(dataset) * 0.8)
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# -----------------------------
# 3) Définir le modèle LSTM
# -----------------------------
class PlankLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, num_classes=1, dropout=0.3):
        super(PlankLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, dropout=dropout if num_layers>1 else 0)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # dernière frame
        out = self.fc(out)
        return out

input_size = X.shape[2]  # nombre de colonnes landmarks
model = PlankLSTM(input_size)
print(model)

# -----------------------------
# 4) Définir la perte et l’optimizer
# -----------------------------
criterion = nn.BCELoss()  # binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# -----------------------------
# 5) Entraînement
# -----------------------------
epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.float().unsqueeze(1).to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    val_loss = 0
    model.eval()
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.float().to(device), yb.float().unsqueeze(1).to(device)
            preds = model(xb)
            loss = criterion(preds, yb)
            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {total_loss/len(train_loader):.4f} - Val Loss: {val_loss/len(val_loader):.4f}")

# -----------------------------
# 6) Sauvegarde du modèle
# -----------------------------
os.makedirs("plank_model/model", exist_ok=True)
torch.save(model.state_dict(), "plank_model/model/plank_model.pth")
print("Modèle sauvegardé → plank_model/model/plank_model.pth")
